In [ ]:
# !pip install langchain_chroma
# !pip install -U langchain-community
# !pip install keybert
# !pip install rank_bm25
# !pip install FlagEmbedding
# !pip install langchain_google_genai

In [ ]:
from langchain_chroma import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
import requests
import json
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter

### Load contents from Notion

In [ ]:
# Notion 인증 정보
NOTION_TOKEN = "API Key"
NOTION_VERSION = "2022-06-28"

headers = {
    "Authorization": f"Bearer {NOTION_TOKEN}",
    "Notion-Version": NOTION_VERSION,
    "Content-Type": "application/json"
}

def get_page_content(page_id):
    """
    페이지 ID를 기반으로 페이지 내용을 가져옵니다.

    Args:
        page_id (str): Notion 페이지 ID (대시 제외)

    Returns:
        dict: 페이지 속성 및 내용
    """
    # 페이지 속성 가져오기
    page_url = f"https://api.notion.com/v1/pages/{page_id}"
    response = requests.get(page_url, headers=headers)

    if response.status_code != 200:
        print(f"오류 발생: {response.status_code}")
        print(response.text)
        return None

    page_data = response.json()

    # 페이지 내용(블록) 가져오기
    blocks_url = f"https://api.notion.com/v1/blocks/{page_id}/children"
    response = requests.get(blocks_url, headers=headers)

    if response.status_code != 200:
        print(f"블록 가져오기 오류: {response.status_code}")
        print(response.text)
        return page_data

    blocks_data = response.json()

    # 페이지 데이터에 블록 내용 추가
    page_data["content"] = blocks_data["results"]

    return page_data

def fetch_all_blocks(block_id):
    """
    블록의 모든 하위 블록을 재귀적으로 가져옵니다.

    Args:
        block_id (str): 블록 ID

    Returns:
        list: 모든 하위 블록 목록
    """
    blocks_url = f"https://api.notion.com/v1/blocks/{block_id}/children"
    response = requests.get(blocks_url, headers=headers)

    if response.status_code != 200:
        print(f"블록 가져오기 오류: {response.status_code}")
        print(response.text)
        return []

    blocks_data = response.json()["results"]

    # 하위 블록이 있는 블록 타입들
    has_children_types = [
        "paragraph", "bulleted_list_item", "numbered_list_item",
        "toggle", "child_page", "child_database", "column_list",
        "column", "table", "synced_block"
    ]

    all_blocks = []

    for block in blocks_data:
        all_blocks.append(block)

        # 하위 블록이 있는 경우 재귀적으로 가져오기
        if block.get("has_children") and block.get("type") in has_children_types:
            children = fetch_all_blocks(block["id"])
            # 하위 블록이 있는 경우에만 children 키 추가
            if children:
                block["children"] = children

    return all_blocks

def extract_text_content(blocks):
    """
    블록에서 텍스트 내용만 추출합니다. 이때, child_page의 제목을 구분자로 먼저 추가합니다.

    Args:
        blocks (list): 블록 목록

    Returns:
        str: 추출된 텍스트 내용
    """
    text_content = ""

    for block in blocks:
        block_type = block.get("type")

        # child_page의 경우, 제목 먼저 삽입
        if block_type == "child_page":
            child_title = block.get("child_page", {}).get("title", "")
            if child_title:
                text_content += f"\n\n### {child_title} ###\n\n"

        if block_type == "paragraph":
            rich_text = block.get("paragraph", {}).get("rich_text", [])
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "heading_1":
            rich_text = block.get("heading_1", {}).get("rich_text", [])
            for text in rich_text:
                text_content += "# " + text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "heading_2":
            rich_text = block.get("heading_2", {}).get("rich_text", [])
            for text in rich_text:
                text_content += "## " + text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "heading_3":
            rich_text = block.get("heading_3", {}).get("rich_text", [])
            for text in rich_text:
                text_content += "### " + text.get("plain_text", "")
            text_content += "\n\n"

        elif block_type == "bulleted_list_item":
            rich_text = block.get("bulleted_list_item", {}).get("rich_text", [])
            text_content += "• "
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n"

        elif block_type == "numbered_list_item":
            rich_text = block.get("numbered_list_item", {}).get("rich_text", [])
            text_content += "1. "
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n"

        elif block_type == "to_do":
            rich_text = block.get("to_do", {}).get("rich_text", [])
            checked = block.get("to_do", {}).get("checked", False)
            text_content += "[{}] ".format("x" if checked else " ")
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n"

        elif block_type == "code":
            rich_text = block.get("code", {}).get("rich_text", [])
            language = block.get("code", {}).get("language", "")
            text_content += f"```{language}\n"
            for text in rich_text:
                text_content += text.get("plain_text", "")
            text_content += "\n```\n\n"

        # 자식 블록이 있으면 재귀적으로 처리
        if "children" in block:
            text_content += extract_text_content(block["children"])

    return text_content


# 메인 실행 코드
def main():
    # 사용자로부터 페이지 ID 입력받기
    page_id = input("Notion 페이지 ID를 입력하세요: ")
    page_id = page_id.replace("-", "")  # 대시가 있으면 제거

    print("페이지 내용을 가져오는 중...")

    # 페이지 내용 가져오기
    page_data = get_page_content(page_id)

    if not page_data:
        print("페이지를 가져올 수 없습니다.")
        return

    # 모든 블록 가져오기 (재귀적으로)
    print("모든 블록을 재귀적으로 가져오는 중...")
    all_blocks = fetch_all_blocks(page_id)

    # 텍스트 내용 추출
    text_content = extract_text_content(all_blocks)

    # 결과 저장
    print("\n=== 페이지 기본 정보 ===")
    page_title = page_data.get("properties", {}).get("title", {}).get("title", [])
    if page_title:
        title_text = " ".join([text.get("plain_text", "") for text in page_title])
        print(f"제목: {title_text}")

    # JSON 형식으로 저장
    with open("chromadb_notion/notion_page_content.json", "w", encoding="utf-8") as f:
        json.dump(page_data, f, ensure_ascii=False, indent=2)

    # 텍스트 형식으로 저장
    with open("chromadb_notion/notion_page_content.txt", "w", encoding="utf-8") as f:
        f.write(text_content)

    print("\n페이지 내용이 'notion_page_content.json'과 'notion_page_content.txt'로 저장되었습니다.")

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
if __name__ == "__main__":
    main()

Notion 페이지 ID를 입력하세요: 1dd124ddd31380b2a22fe97dc9b0539e
페이지 내용을 가져오는 중...
모든 블록을 재귀적으로 가져오는 중...

=== 페이지 기본 정보 ===
제목: 공산주의자가 온다

페이지 내용이 'notion_page_content.json'과 'notion_page_content.txt'로 저장되었습니다.


### Split Text

In [ ]:
import re
from keybert import KeyBERT
from transformers import BertModel

# 텍스트 로드 함수
def load_text_from_file(file_path):
    with open(file_path, "r", encoding="utf-8") as file:
        return file.read()

# 상위 구분자 ("### 제목 ###") 기준으로 분할
def split_text_with_headers(text):
    # 헤더 패턴을 더 유연하게
    chunks = re.split(r"###\s*(.*?)\s*###", text)
    result = []
    if len(chunks) == 1:  # 헤더가 없을 경우 전체를 하나로
        return [("전체", text)]
    for i in range(1, len(chunks), 2):
        title = chunks[i].strip()
        content = chunks[i + 1].strip() if i + 1 < len(chunks) else ""
        result.append((title, content))
    return result

# 상위 구분자 기준 → 하위에서 RecursiveCharacterTextSplitter 적용
def chunk_text_with_recursive_splitter(text, chunk_size=500, chunk_overlap=50):
    header_chunks = split_text_with_headers(text)
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap
    )
    final_chunks = []

    for title, content in header_chunks:
        if not content.strip():
            continue
        temp_doc = Document(page_content=content, metadata={"title": title})
        sub_chunks = text_splitter.split_documents([temp_doc])

        print(f"제목: {title}, 내용 길이: {len(content)}, 청크 개수: {len(sub_chunks)}")  # 🔍 디버깅 출력

        for sub_chunk in sub_chunks:
            final_chunks.append(sub_chunk)

    return final_chunks

# .txt 파일 경로
txt_file_path = "/chromadb_notion/notion_page_content.txt"

# 텍스트 로드
text = load_text_from_file(txt_file_path)

# 청킹 실행
texts = chunk_text_with_recursive_splitter(text, chunk_size=800)

제목: 전체, 내용 길이: 2421, 청크 개수: 4


In [ ]:
text

"1️⃣ 한줄\xa0평\n\n소설가는 도핑 테스트 안 하나?\n\n♓ Inuit Points ★★★★☆\n\n한국형 SF 소설들 모음입니다. 상상력이 기발하여 찬탄하며 읽게 됩니다. 시종 경쾌한 문체속에 묵직한 생각도 언뜻언뜻 비쳐보입니다. 세계관이 발랄하다보니, 독자는 감도 못잡고 서사를 따라가다 롤러코스터를 타다, 시야가 전복되는 결말에 어질어질 이야기를 마무리합니다. 김동식을 봤을 때의 충격에 김애란을 접했을 때의 경이가 중첩된 기시감을 느꼈습니다. 지금도 이미 좋지만, 10년 후엔 더 큰 작가가 될지도 모른다는 생각을 했습니다.\n\n🎢 Stories Related\n\n• 이신주 작가는 96년생으로 상당히 젊지만, 글 쓴지는 꽤 오래됩니다.\n• 2018 한국과학문학상을 비롯, 해당 부문 대상을 수차례 수상하며 두각을 드러냈습니다.\n• 첫 습작은 고등학교 수업시간에 상상하다, 분필이 화자인 글이라고 합니다.\n이신주 2023\n\n🗨️ 좀\xa0더\xa0자세한\xa0이야기\n\n2025년 4월 기준, 나무위키 페이지도 아직 없는 신예\xa0작가입니다. 누가 강추하길래 장바구니에 넣어두고도, 제목이 흥미롭지 않아 미루다가 읽었습니다. 한 두편 읽다 보니, 의구심이 놀라움으로 바뀌고 갈수록 흥미진진하게 읽게 되었습니다.\n\n12편 모두 SF답게 기이한 세계관입니다. SF 중 제가 좋아하는 테드 창이나 켄 리우는 압도적 과학 이론을 기반으로 이야기를 풀어나간다면, 한국 SF는 판타지 성향이 강합니다. 서너 문장 정도로 갈음되는 간단한 과학적 장치로 빠르게 세계관을 구성하고, 그 안의 상호작용을 오래 이야기하지요. 이신주도 그러합니다.\n\n예컨대, 무슨 물질이든 먹을 수 있게 바꿔주는 물질이 생긴다면 어떨까. 결국 맛의 추구가 극에 달하면, 자기를 먹는 자기애적 자가포식까지 가기도 합니다. 더 나가면 신의 맛을 궁금해 하는 사람들도 나오죠. 즉, 과학적 엄밀성을 버릴 때, 우리의 본능이 치닫는 그곳, 모종의 심리학적 중력을 벗어난 새로운 심리법칙에 따른

In [ ]:
texts

[Document(metadata={'title': '전체'}, page_content='1️⃣ 한줄\xa0평\n\n소설가는 도핑 테스트 안 하나?\n\n♓ Inuit Points ★★★★☆\n\n한국형 SF 소설들 모음입니다. 상상력이 기발하여 찬탄하며 읽게 됩니다. 시종 경쾌한 문체속에 묵직한 생각도 언뜻언뜻 비쳐보입니다. 세계관이 발랄하다보니, 독자는 감도 못잡고 서사를 따라가다 롤러코스터를 타다, 시야가 전복되는 결말에 어질어질 이야기를 마무리합니다. 김동식을 봤을 때의 충격에 김애란을 접했을 때의 경이가 중첩된 기시감을 느꼈습니다. 지금도 이미 좋지만, 10년 후엔 더 큰 작가가 될지도 모른다는 생각을 했습니다.\n\n🎢 Stories Related\n\n• 이신주 작가는 96년생으로 상당히 젊지만, 글 쓴지는 꽤 오래됩니다.\n• 2018 한국과학문학상을 비롯, 해당 부문 대상을 수차례 수상하며 두각을 드러냈습니다.\n• 첫 습작은 고등학교 수업시간에 상상하다, 분필이 화자인 글이라고 합니다.\n이신주 2023\n\n🗨️ 좀\xa0더\xa0자세한\xa0이야기\n\n2025년 4월 기준, 나무위키 페이지도 아직 없는 신예\xa0작가입니다. 누가 강추하길래 장바구니에 넣어두고도, 제목이 흥미롭지 않아 미루다가 읽었습니다. 한 두편 읽다 보니, 의구심이 놀라움으로 바뀌고 갈수록 흥미진진하게 읽게 되었습니다.\n\n12편 모두 SF답게 기이한 세계관입니다. SF 중 제가 좋아하는 테드 창이나 켄 리우는 압도적 과학 이론을 기반으로 이야기를 풀어나간다면, 한국 SF는 판타지 성향이 강합니다. 서너 문장 정도로 갈음되는 간단한 과학적 장치로 빠르게 세계관을 구성하고, 그 안의 상호작용을 오래 이야기하지요. 이신주도 그러합니다.'),
 Document(metadata={'title': '전체'}, page_content="예컨대, 무슨 물질이든 먹을 수 있게 바꿔주는 물질이 생긴다면 어떨까. 결국 맛의 추구가 극에 달하면, 자기를 먹는 자기애적 자가포식까지 가기

In [ ]:
# 출력
for i, chunk in enumerate(texts):
    num_chars = len(chunk)  # ✅ 글자 수 세기
    print("-" * 100)
    print(f"[{i+1}번째 청크] (글자 수: {num_chars})")
    print(chunk)

### Save as vector DB with ChromaDB

In [ ]:
embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")

vector_db = Chroma.from_documents(
    documents=texts,
    embedding=embedding_model,
    persist_directory="./chroma_db"
)

In [ ]:
# load from disk
loaded_vector_db = Chroma(persist_directory="./chroma_db", embedding_function=embedding_model)

### Retrieve chunks with BM25 + similarity

In [ ]:
from rank_bm25 import BM25Okapi
from transformers import AutoTokenizer
from sklearn.metrics.pairwise import cosine_similarity
import re
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
from FlagEmbedding import FlagReranker

# Hugging Face tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained("jhgan/ko-sbert-sts")

# BGE 리랭커 불러오기 (cross-encoder 방식)
reranker = FlagReranker("BAAI/bge-reranker-base", use_fp16=True)

def hybrid_search_with_reranker(query, vector_db, documents, k=2, bm25_weight=0.5):
    """
    Hybrid search (BM25 + Cosine) + BGE Reranker

    Parameters:
    - query: 검색 쿼리
    - vector_db: 코사인 유사도 검색용 벡터 데이터베이스
    - documents: 원본 문서 리스트
    - k: 최종 반환할 결과 수
    - bm25_weight: BM25 가중치 (0-1), 나머지는 코사인 유사도 가중치

    Returns:
    - reranked 결과 리스트
    """
    # -----------------------
    # 1. 코사인 유사도 검색 (벡터 검색)
    # -----------------------
    cosine_results = vector_db.similarity_search_with_score(query, k=k*3)  # 후보 넉넉히 확보
    cosine_docs = [doc for doc, score in cosine_results]
    cosine_scores = [score for doc, score in cosine_results]

    max_score = max(cosine_scores)
    min_score = min(cosine_scores)
    score_range = max_score - min_score if max_score != min_score else 1
    normalized_cosine_scores = [1 - ((score - min_score) / score_range) for score in cosine_scores]

    # -----------------------
    # 2. BM25 준비
    # -----------------------
    tokenized_docs = [tokenizer.tokenize(doc.page_content) for doc in documents]
    bm25 = BM25Okapi(tokenized_docs)

    tokenized_query = tokenizer.tokenize(query)
    bm25_scores = bm25.get_scores(tokenized_query)

    max_bm25 = max(bm25_scores)
    min_bm25 = min(bm25_scores)
    bm25_range = max_bm25 - min_bm25 if max_bm25 != min_bm25 else 1
    normalized_bm25_scores = [(score - min_bm25) / bm25_range for score in bm25_scores]

    # -----------------------
    # 3. 하이브리드 점수 계산
    # -----------------------
    hybrid_scores = []
    for i, doc in enumerate(documents):
        cosine_score = 0
        for j, cosine_doc in enumerate(cosine_docs):
            if doc.page_content == cosine_doc.page_content:
                cosine_score = normalized_cosine_scores[j]
                break
        hybrid_score = (bm25_weight * normalized_bm25_scores[i]) + ((1 - bm25_weight) * cosine_score)
        hybrid_scores.append((doc, hybrid_score))

    # 후보군 (BM25 + Cosine top-k*3)
    hybrid_results = sorted(hybrid_scores, key=lambda x: x[1], reverse=True)[:k*3]
    candidate_docs = [doc for doc, score in hybrid_results]

    # -----------------------
    # 4. BGE 리랭커 적용
    # -----------------------
    pairs = [[query, doc.page_content] for doc in candidate_docs]
    scores = reranker.compute_score(pairs)  # relevance score

    reranked_docs = [
        doc for _, doc in sorted(zip(scores, candidate_docs), key=lambda x: x[0], reverse=True)
    ]

    # 최종 top-k 결과 반환
    return reranked_docs[:k]

tokenizer_config.json:   0%|          | 0.00/443 [00:00<?, ?B/s]

sentencepiece.bpe.model:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/17.1M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/279 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/799 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.11G [00:00<?, ?B/s]

In [ ]:
# Hugging Face tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained("jhgan/ko-sbert-sts")

def hybrid_search(query, vector_db, documents, k=2, bm25_weight=0.5):
    """
    Hybrid search combining BM25 and cosine similarity with customizable weighting

    Parameters:
    - query: 검색 쿼리
    - vector_db: 코사인 유사도 검색용 벡터 데이터베이스
    - documents: 원본 문서 리스트
    - k: 반환할 결과 수
    - bm25_weight: BM25 가중치 (0-1), 나머지는 코사인 유사도 가중치

    Returns:
    - 최종 결과 리스트
    """
    # 1. 코사인 유사도 검색 (벡터 검색)
    cosine_results = vector_db.similarity_search_with_score(query, k=k*2)

    cosine_docs = [doc for doc, score in cosine_results]
    cosine_scores = [score for doc, score in cosine_results]

    max_score = max(cosine_scores)
    min_score = min(cosine_scores)
    score_range = max_score - min_score if max_score != min_score else 1
    normalized_cosine_scores = [1 - ((score - min_score) / score_range) for score in cosine_scores]

    # 2. BM25 준비 (Hugging Face tokenizer 기반)
    tokenized_docs = [tokenizer.tokenize(doc.page_content) for doc in documents]
    bm25 = BM25Okapi(tokenized_docs)

    tokenized_query = tokenizer.tokenize(query)
    bm25_scores = bm25.get_scores(tokenized_query)

    max_bm25 = max(bm25_scores)
    min_bm25 = min(bm25_scores)
    bm25_range = max_bm25 - min_bm25 if max_bm25 != min_bm25 else 1
    normalized_bm25_scores = [(score - min_bm25) / bm25_range for score in bm25_scores]

    # 3. 하이브리드 점수 계산
    hybrid_scores = []
    for i, doc in enumerate(documents):
        cosine_score = 0
        for j, cosine_doc in enumerate(cosine_docs):
            if doc.page_content == cosine_doc.page_content:
                cosine_score = normalized_cosine_scores[j]
                break
        hybrid_score = (bm25_weight * normalized_bm25_scores[i]) + ((1 - bm25_weight) * cosine_score)
        hybrid_scores.append((doc, hybrid_score))

    hybrid_results = sorted(hybrid_scores, key=lambda x: x[1], reverse=True)[:k]
    return [doc for doc, score in hybrid_results]

In [ ]:
# 쿼리 실행
query = "단일성 정체감의 장애 현상은 어떻게 나타나는가?"

# Re-create the vector database as it failed in a previous step
embedding_model = HuggingFaceEmbeddings(model_name="jhgan/ko-sbert-sts")

# Check if texts is empty before creating the vector database
if texts:
    vector_db = Chroma.from_documents(
        documents=texts,
        embedding=embedding_model,
        persist_directory="./chroma_db"
    )
else:
    print("No text chunks available to create the vector database.")
    vector_db = None # Set vector_db to None if creation fails


# 하이브리드 검색 실행 (BM25 0%, 코사인 유사도 100%)
if vector_db:
    results = hybrid_search_with_reranker(query, vector_db, texts, k=3, bm25_weight=0.5)

    # 결과 출력
    for result in results:
        print(result.page_content, "\n\n")
else:
    print("Vector database not created. Cannot perform hybrid search.")

You're using a XLMRobertaTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


단일성 정체감의 장애 현상은 이렇습니다. 인격이 하나라 세상 모든 일을 합리성이라는 단일 필터로 본다는 점, 그래서 남의 마음을 잘 이해 못한다는 점입니다. 인격이 하나라서 현실에 과몰입할 뿐더러, 사랑에 집착하여 통제욕구가 강한 장애인들입니다. 자연, 결혼도 법률 관계 이상으로 과하게 상상하며, 자식들에게도 일관되라고 자꾸 단일 인격을 강요한다고 보고되었다 합니다. 그래서 '우리와 같은' 사람으로 존중하지만, 절대 결혼하진 말라고 권합니다.

이처럼 모든게 지금 세상과 같지만, 특정 부분이 살짝 비틀어진 세계를 상상하면서, 지금 우리가 가진 것들을 재음미합니다. 쉽게 보면 인생의 패러디고 위트있는 냉소지만, 곰곰 생각하면 통찰적입니다. 인격과 삶에서 일관성의 장점만 생각했지 단점이 있을 수도 있다는걸 깨닫기 어려웠던 것 처럼 말입니다.

책은 매우 기묘하지만, 지적, 정서적 쾌감이 있는 이야기들로 빼곡합니다. 인터뷰에서 읽었던 이신주 작가의 독특한 글쓰기 습관이 있는데, 글감을 찾기 위해 랜덤 단어를 돌려 그걸로 글쓰기를 꾸준히 한다고 합니다. 그런데 마지막 저자 후기에 씨앗이 되는 단어들을 적어둔게 매우 귀합니다.

사실 저 기묘한 조합으로 문장하나 쓰기도 어렵게 느껴집니다. 이신주의 글을 읽고 난 이후에도 저 단어는 씨앗일 뿐 신선하고 천연덕스러운 이야기입니다. 어찌보면, 기묘한 세계관을 상상하는데는 저런 지나치게 임의적 단어를 억지로 이어붙이는 제약 때문에 창의가 샘솟겠지요. 그래서 그의 생각 비결을 슬쩍 엿본건 추가의 소득입니다. 


아직은 그의 문장이 매끄럽거나 유려하진 않습니다. 문득 비치는 언어적 가벼움은 장점이지만, 묘사나 이야기를 쌓아가는 구조는 최고 작가에 비해선 평범합니다. 그럼에도 저는 그의 향후를 매우 밝게 봅니다. 문장력은 수련의 영역이고 경이로운 스토리 메이킹은 보다 본원적이니까요. 바라자면, 과학을 사용한 허구의 현실성을 깊게 하고, 이야기 자체를 촘촘히 쌓아나가는 능력을 겸비할 수 있다면, 몇 십년 쯤 후 우리는 세계적 SF 

### Hyde + query decomposition

In [ ]:
import google.generativeai as genai
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

from langchain.llms.base import LLM
from typing import Any, List, Mapping, Optional

In [ ]:
genai.configure(api_key="Your API Key")
GEMINI_MODEL = 'gemini-2.0-flash'
GENAI_TEMP = 0.5

In [ ]:
def generate_gemini_answer(query: str, context: str, model_name=GEMINI_MODEL, temperature=GENAI_TEMP) -> str:
    prompt = f"Context:\n{context}\n\nQuestion: {query}"
    model = genai.GenerativeModel(
        model_name,
        generation_config=genai.GenerationConfig(temperature=temperature)
    )
    return model.generate_content(prompt).text

In [ ]:
# 쿼리 실행
question = "'공생' 개념의 의미와 중요성은 무엇인가요?"
# question = "‘잊혀질 권리’에 대해 '우리가 두고 온 100가지 유실물' 저자는 어떤 시선을 보여주고 있나요?"

query = generate_gemini_answer(question,
                               "Question에 대한 요약된 답변을 출력해줘.",
                               GEMINI_MODEL,
                               GENAI_TEMP)

print(query)

공생은 서로 다른 종의 생물들이 서로에게 이익을 주거나, 한쪽은 이익을 얻고 다른 쪽은 해를 입지 않는 관계를 의미합니다. 이러한 공생 관계는 생태계의 안정성과 종의 생존에 중요한 역할을 합니다.



In [ ]:
results = hybrid_search(query, loaded_vector_db, texts, k=3, bm25_weight=0.5)

# 결과 출력
for result in results:
    print(result.page_content, "\n\n")

예컨대, 무슨 물질이든 먹을 수 있게 바꿔주는 물질이 생긴다면 어떨까. 결국 맛의 추구가 극에 달하면, 자기를 먹는 자기애적 자가포식까지 가기도 합니다. 더 나가면 신의 맛을 궁금해 하는 사람들도 나오죠. 즉, 과학적 엄밀성을 버릴 때, 우리의 본능이 치닫는 그곳, 모종의 심리학적 중력을 벗어난 새로운 심리법칙에 따른 행동들은 다시 지금 세계를 돌이켜 보는 좋은 대척점이 됩니다. 이게 SF나 환상소설의 미덕이라고 저는 생각합니다.

책에서 제가 가장 좋아하는 작품을 들어 좀 더 설명하겠습니다.

다행히 이 작품은 스포일러가 없습니다. '단일성 정체감 장애'를 다룹니다. 의학적 설명문의 형식을 취해 읽다보면 경이롭습니다. 즉, 이 설명문의 독자층은 작중 일반인, 5~6개의 다중인격을 가진 세상입니다. 여기에선 지금의 우리 같은 하나의 인격, 일관된 인격을 가진 사람이 장애로 취급받습니다. 그리고 글은, 흔한 의학적 책처럼 PC하게 쓰여집니다. 단일성 정체성을 가진 사람들도 우리와 같은 사람이니 똑같은 사랑을 받을 자격이 있고, 무서워하지 말되 그들의 특성을 알고 조심히 접근하라고 합니다. 


단일성 정체감의 장애 현상은 이렇습니다. 인격이 하나라 세상 모든 일을 합리성이라는 단일 필터로 본다는 점, 그래서 남의 마음을 잘 이해 못한다는 점입니다. 인격이 하나라서 현실에 과몰입할 뿐더러, 사랑에 집착하여 통제욕구가 강한 장애인들입니다. 자연, 결혼도 법률 관계 이상으로 과하게 상상하며, 자식들에게도 일관되라고 자꾸 단일 인격을 강요한다고 보고되었다 합니다. 그래서 '우리와 같은' 사람으로 존중하지만, 절대 결혼하진 말라고 권합니다.

이처럼 모든게 지금 세상과 같지만, 특정 부분이 살짝 비틀어진 세계를 상상하면서, 지금 우리가 가진 것들을 재음미합니다. 쉽게 보면 인생의 패러디고 위트있는 냉소지만, 곰곰 생각하면 통찰적입니다. 인격과 삶에서 일관성의 장점만 생각했지 단점이 있을 수도 있다는걸 깨닫기 어려웠던 것 처럼 말입니다.

책은 매우 기묘하지만, 지적, 정

In [ ]:
from transformers import AutoTokenizer
from rank_bm25 import BM25Okapi
from FlagEmbedding import FlagReranker

# Hugging Face tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained("jhgan/ko-sbert-sts")

# BGE 리랭커 불러오기 (cross-encoder 방식)
reranker = FlagReranker("BAAI/bge-reranker-base", use_fp16=True)

def hybrid_search_with_reranker(query, vector_db, documents, k=2, bm25_weight=0.5):
    """
    Hybrid search (BM25 + Cosine) + BGE Reranker

    Parameters:
    - query: 검색 쿼리
    - vector_db: 코사인 유사도 검색용 벡터 데이터베이스
    - documents: 원본 문서 리스트
    - k: 최종 반환할 결과 수
    - bm25_weight: BM25 가중치 (0-1), 나머지는 코사인 유사도 가중치

    Returns:
    - reranked 결과 리스트
    """
    # -----------------------
    # 1. 코사인 유사도 검색 (벡터 검색)
    # -----------------------
    cosine_results = vector_db.similarity_search_with_score(query, k=k*3)  # 후보 넉넉히 확보
    cosine_docs = [doc for doc, score in cosine_results]
    cosine_scores = [score for doc, score in cosine_results]

    max_score = max(cosine_scores)
    min_score = min(cosine_scores)
    score_range = max_score - min_score if max_score != min_score else 1
    normalized_cosine_scores = [1 - ((score - min_score) / score_range) for score in cosine_scores]

    # -----------------------
    # 2. BM25 준비
    # -----------------------
    tokenized_docs = [tokenizer.tokenize(doc.page_content) for doc in documents]
    bm25 = BM25Okapi(tokenized_docs)

    tokenized_query = tokenizer.tokenize(query)
    bm25_scores = bm25.get_scores(tokenized_query)

    max_bm25 = max(bm25_scores)
    min_bm25 = min(bm25_scores)
    bm25_range = max_bm25 - min_bm25 if max_bm25 != min_bm25 else 1
    normalized_bm25_scores = [(score - min_bm25) / bm25_range for score in bm25_scores]

    # -----------------------
    # 3. 하이브리드 점수 계산
    # -----------------------
    hybrid_scores = []
    for i, doc in enumerate(documents):
        cosine_score = 0
        for j, cosine_doc in enumerate(cosine_docs):
            if doc.page_content == cosine_doc.page_content:
                cosine_score = normalized_cosine_scores[j]
                break
        hybrid_score = (bm25_weight * normalized_bm25_scores[i]) + ((1 - bm25_weight) * cosine_score)
        hybrid_scores.append((doc, hybrid_score))

    # 후보군 (BM25 + Cosine top-k*3)
    hybrid_results = sorted(hybrid_scores, key=lambda x: x[1], reverse=True)[:k*3]
    candidate_docs = [doc for doc, score in hybrid_results]

    # -----------------------
    # 4. BGE 리랭커 적용
    # -----------------------
    pairs = [[query, doc.page_content] for doc in candidate_docs]
    scores = reranker.compute_score(pairs)  # relevance score

    reranked_docs = [
        doc for _, doc in sorted(zip(scores, candidate_docs), key=lambda x: x[0], reverse=True)
    ]

    # 최종 top-k 결과 반환
    return reranked_docs[:k]

In [ ]:
from transformers import AutoTokenizer
from rank_bm25 import BM25Okapi
from FlagEmbedding import FlagReranker

# Hugging Face tokenizer 불러오기
tokenizer = AutoTokenizer.from_pretrained("jhgan/ko-sbert-sts")

# BGE 리랭커 불러오기 (cross-encoder 방식)
reranker = FlagReranker("BAAI/bge-reranker-base", use_fp16=True)

def hybrid_search_with_reranker(query, vector_db, documents, k=2, bm25_weight=0.5):
    """
    Hybrid search (BM25 + Cosine) + BGE Reranker

    Parameters:
    - query: 검색 쿼리
    - vector_db: 코사인 유사도 검색용 벡터 데이터베이스
    - documents: 원본 문서 리스트
    - k: 최종 반환할 결과 수
    - bm25_weight: BM25 가중치 (0-1), 나머지는 코사인 유사도 가중치

    Returns:
    - reranked 결과 리스트
    """
    # -----------------------
    # 1. 코사인 유사도 검색 (벡터 검색)
    # -----------------------
    cosine_results = vector_db.similarity_search_with_score(query, k=k*3)  # 후보 넉넉히 확보
    cosine_docs = [doc for doc, score in cosine_results]
    cosine_scores = [score for doc, score in cosine_results]

    max_score = max(cosine_scores)
    min_score = min(cosine_scores)
    score_range = max_score - min_score if max_score != min_score else 1
    normalized_cosine_scores = [1 - ((score - min_score) / score_range) for score in cosine_scores]

    # -----------------------
    # 2. BM25 준비
    # -----------------------
    tokenized_docs = [tokenizer.tokenize(doc.page_content) for doc in documents]
    bm25 = BM25Okapi(tokenized_docs)

    tokenized_query = tokenizer.tokenize(query)
    bm25_scores = bm25.get_scores(tokenized_query)

    max_bm25 = max(bm25_scores)
    min_bm25 = min(bm25_scores)
    bm25_range = max_bm25 - min_bm25 if max_bm25 != min_bm25 else 1
    normalized_bm25_scores = [(score - min_bm25) / bm25_range for score in bm25_scores]

    # -----------------------
    # 3. 하이브리드 점수 계산
    # -----------------------
    hybrid_scores = []
    for i, doc in enumerate(documents):
        cosine_score = 0
        for j, cosine_doc in enumerate(cosine_docs):
            if doc.page_content == cosine_doc.page_content:
                cosine_score = normalized_cosine_scores[j]
                break
        hybrid_score = (bm25_weight * normalized_bm25_scores[i]) + ((1 - bm25_weight) * cosine_score)
        hybrid_scores.append((doc, hybrid_score))

    # 후보군 (BM25 + Cosine top-k*3)
    hybrid_results = sorted(hybrid_scores, key=lambda x: x[1], reverse=True)[:k*3]
    candidate_docs = [doc for doc, score in hybrid_results]

    # -----------------------
    # 4. BGE 리랭커 적용
    # -----------------------
    pairs = [[query, doc.page_content] for doc in candidate_docs]
    scores = reranker.compute_score(pairs)  # relevance score

    reranked_docs = [
        doc for _, doc in sorted(zip(scores, candidate_docs), key=lambda x: x[0], reverse=True)
    ]

    # 최종 top-k 결과 반환
    return reranked_docs[:k]


